## Задача о движении 2 небесных тел в СС

Задача о движении 2 небесных тел является распростанненой задачей механики. 

Пусть есть 2 тела, имеющих массы $m_1$ и $m_2$ соотвественно. Напомним Второй закон Ньютона
$$F = ma,$$
где $a$ это ускорение. Тогда сила притяжения тела 1 на тело 2 запишется в виде
$$F_{21}=\frac{Gm_{1}m_{2}}{r^3_{21}}r_{21},$$
где $r_{21}$ это вектор расстояния между двумя телам. Сила, действующая на из тела 2 на тело 1, по Третьему закону  Ньютона равна $F_{12}=−F_{21}$, а расстояние $r_{12}=−r_{21}$. 
Данная задача описывается следующей системой дифференциальных уравнений: $$\begin{cases}
x^{″}_1(t)=\frac{Gm_2(x_2−x_1)}{|r|^3}\\
y^{″}_1(t)=\frac{Gm_2(y_2−y_1)}{|r|^3}\\
x^{″}_2(t)=\frac{Gm_1(x_1−x_2)}{|r|^3}\\
y^{″}_2(t)=\frac{Gm_1(y_1−y_2)}{|r|^3}\\
\end{cases}$$

Данную систему можно свести к систему из 8 ДУ первого порядка и решить численно с помощью Метода Рунге-Кутта.

Класс для решения СДУ методом Рунге-Кутта 4 порядка:

In [1]:
from numpy import *

# The gravitational constant G
G = 6.67428e-11

class RK4():
    def __init__(self, r1, r2, v1, v2, step, m1, m2):
        self.r1 = r1
        self.r2 = r2
        self.v1 = v1
        self.v2 = v2
        self.h = step
        self.m1 = m1
        self.m2 = m2
        self.t = 0
        
    def getR1(self):
        return self.r1
    
    def getR2(self):
        return self.r2
    
    def getV1(self):
        return self.v1
    
    def getV2(self):
        return self.v2
    
    def getH(self):
        return self.h
    
    def getM1(self):
        return self.m1
    
    def getT(self):
        return self.t
    
    ############
    def __gravityForce(self, s1, s2, mass):
        return ((s2 - s1) * G * mass) / (linalg.norm(s2 - s1) ** 3)
    
    def __getAccBody(self, r1, r2, m):
        return ((r2 - r1) * G * m) / (linalg.norm(r2 - r1)**3) # Ускорение первого тела
    ############
    
    def runIter(self):
        g1 = self.v1
        g2 = self.v1 + g1 * 0.5 * self.h
        g3 = self.v1 + g2 * 0.5 * self.h
        g4 = self.v1 + g3 * self.h
        c = (g1 + (g2 * 2) + (g3 * 2) + g4) * self.h / 6.0 + self.r1
        
        a1 = self.__getAccBody(self.r1, self.r2, self.m2)
        a2 = self.__getAccBody(self.r2, self.r1, self.m1)
        
        h1 = self.v2
        h2 = self.v2 + h1 * 0.5 * self.h
        h3 = self.v2 + h2 * 0.5 * self.h
        h4 = self.v2 + h3 * self.h
        d = (h1 + h2 * 2 + h3 * 2 + h4) * self.h / 6.0 + self.r2

        k1 = self.__gravityForce(self.r1, self.r2, self.m2)
        k2 = self.__gravityForce(self.r1 + g1 * 0.5 * self.h, self.r2 + h1 * 0.5 * self.h, self.m2)
        k3 = self.__gravityForce(self.r1 + g2 * 0.5 * self.h, self.r2 + h2 * 0.5 * self.h, self.m2)
        k4 = self.__gravityForce(self.r1 + g2 * self.h, self.r2 + h2 * self.h, self.m2)
        e = (k1 + k2 * 2 + k3 * 2 + k4) * self.h / 6.0 + self.v1 # Скорость первого тела

        l1 = k1 * (-1)
        l2 = k2 * (-1)
        l3 = k3 * (-1)
        l4 = k4 * (-1)
        f = (l1 + l2 * 2 + l3 * 2 + l4) * self.h / 6.0 + self.v2 #Скорость второго тела

        self.r1 = c
        self.r2 = d
        self.v1 = e
        self.v2 = f
        self.t += self.h

Пример численного решения:

In [5]:
%%time
TBP = RK4(array([0, 0]), array([384000000, 0]), array([0,0]), array([-10, 10]),
                 0.01, 73400000000000000000000.0, 6000000000000000000000000.0)
e = [[],[]]
c = [[],[]]
for i in range(100000):
    print('t = %s' % (TBP.getT()))
    print('Earth x = %s, y = %s' %(TBP.getR1()[0], TBP.getR1()[1]))
    print('Moon x = %s, y = %s' %(TBP.getR2()[0], TBP.getR2()[1]))
    print('dX = %s, dY = %s' %(TBP.getR2()[0]-TBP.getR1()[0], TBP.getR2()[1]-TBP.getR1()[1]))
    TBP.runIter()

t = 0
Earth x = 0, y = 0
Moon x = 384000000, y = 0
dX = 384000000, dY = 0
t = 0.01
Earth x = 0.0, y = 0.0
Moon x = 383999999.899, y = 0.100501670833
dX = 383999999.899, dY = 0.100501670833
t = 0.02
Earth x = 2.72939571853e-07, y = 3.56574701168e-17
Moon x = 383999999.799, y = 0.201003341667
dX = 383999999.799, dY = 0.201003341667
t = 0.03
Earth x = 8.188187157e-07, y = 1.7840700161e-16
Moon x = 383999999.698, y = 0.3015050125
dX = 383999999.698, dY = 0.3015050125
t = 0.04
Earth x = 1.63763743169e-06, y = 4.9968318585e-16
Moon x = 383999999.598, y = 0.402006683333
dX = 383999999.598, dY = 0.402006683333
t = 0.05
Earth x = 2.72939571995e-06, y = 1.07092061432e-15
Moon x = 383999999.497, y = 0.502508354167
dX = 383999999.497, dY = 0.502508354167
t = 0.060000000000000005
Earth x = 4.09409358065e-06, y = 1.96355387862e-15
Moon x = 383999999.397, y = 0.603010025
dX = 383999999.397, dY = 0.603010025
t = 0.07
Earth x = 5.7317310139e-06, y = 3.24901757046e-15
Moon x = 383999999.296, y = 0.70351

Moon x = 383999991.054, y = 8.94464870416
dX = 383999991.053, dY = 8.94464870415
t = 0.9000000000000006
Earth x = 0.00109312300206, y = 8.53494423564e-12
Moon x = 383999990.954, y = 9.04515037499
dX = 383999990.953, dY = 9.04515037498
t = 0.9100000000000006
Earth x = 0.00111768756409, y = 8.82424895912e-12
Moon x = 383999990.853, y = 9.14565204582
dX = 383999990.852, dY = 9.14565204582
t = 0.9200000000000006
Earth x = 0.00114252506572, y = 9.12001845374e-12
Moon x = 383999990.753, y = 9.24615371666
dX = 383999990.752, dY = 9.24615371665
t = 0.9300000000000006
Earth x = 0.00116763550693, y = 9.42232415409e-12
Moon x = 383999990.652, y = 9.34665538749
dX = 383999990.651, dY = 9.34665538748
t = 0.9400000000000006
Earth x = 0.00119301888772, y = 9.73123749478e-12
Moon x = 383999990.552, y = 9.44715705832
dX = 383999990.55, dY = 9.44715705831
t = 0.9500000000000006
Earth x = 0.0012186752081, y = 1.00468299104e-11
Moon x = 383999990.451, y = 9.54765872916
dX = 383999990.45, dY = 9.5476587291

Earth x = 0.0191398887224, y = 6.25329842967e-10
Moon x = 383999962.293, y = 37.6881265619
dX = 383999962.274, dY = 37.6881265612
t = 3.759999999999964
Earth x = 0.0192422410719, y = 6.30352566209e-10
Moon x = 383999962.192, y = 37.7886282327
dX = 383999962.173, dY = 37.7886282321
t = 3.7699999999999636
Earth x = 0.019344866361, y = 6.35402113088e-10
Moon x = 383999962.092, y = 37.8891299035
dX = 383999962.072, dY = 37.8891299029
t = 3.7799999999999634
Earth x = 0.0194477645897, y = 6.40478555039e-10
Moon x = 383999961.991, y = 37.9896315744
dX = 383999961.971, dY = 37.9896315737
t = 3.789999999999963
Earth x = 0.0195509357581, y = 6.45581963496e-10
Moon x = 383999961.89, y = 38.0901332452
dX = 383999961.871, dY = 38.0901332445
t = 3.799999999999963
Earth x = 0.0196543798661, y = 6.50712409894e-10
Moon x = 383999961.79, y = 38.190634916
dX = 383999961.77, dY = 38.1906349154
t = 3.8099999999999627
Earth x = 0.0197580969137, y = 6.55869965668e-10
Moon x = 383999961.689, y = 38.2911365868

Earth x = 0.0760494357405, y = 4.95273646846e-09
Moon x = 383999924.849, y = 75.0747481075
dX = 383999924.773, dY = 75.0747481026
t = 7.479999999999885
Earth x = 0.0762533216405, y = 4.97266700396e-09
Moon x = 383999924.748, y = 75.1752497784
dX = 383999924.672, dY = 75.1752497734
t = 7.489999999999885
Earth x = 0.0764574804802, y = 4.99265093679e-09
Moon x = 383999924.648, y = 75.2757514492
dX = 383999924.571, dY = 75.2757514442
t = 7.4999999999998845
Earth x = 0.0766619122596, y = 5.01268833838e-09
Moon x = 383999924.547, y = 75.37625312
dX = 383999924.47, dY = 75.376253115
t = 7.509999999999884
Earth x = 0.0768666169787, y = 5.03277928017e-09
Moon x = 383999924.446, y = 75.4767547908
dX = 383999924.37, dY = 75.4767547858
t = 7.519999999999884
Earth x = 0.0770715946374, y = 5.05292383359e-09
Moon x = 383999924.346, y = 75.5772564616
dX = 383999924.269, dY = 75.5772564566
t = 7.529999999999884
Earth x = 0.0772768452359, y = 5.07312207008e-09
Moon x = 383999924.245, y = 75.6777581324
d

Earth x = 0.17072946725, y = 1.66595937585e-08
Moon x = 383999887.368, y = 112.461369646
dX = 383999887.197, dY = 112.461369629
t = 11.199999999999806
Earth x = 0.17103488672, y = 1.67043175224e-08
Moon x = 383999887.267, y = 112.561871317
dX = 383999887.096, dY = 112.5618713
t = 11.209999999999805
Earth x = 0.17134057913, y = 1.67491212572e-08
Moon x = 383999887.166, y = 112.662372987
dX = 383999886.995, dY = 112.662372971
t = 11.219999999999805
Earth x = 0.17164654448, y = 1.67940050346e-08
Moon x = 383999887.065, y = 112.762874658
dX = 383999886.894, dY = 112.762874641
t = 11.229999999999805
Earth x = 0.17195278277, y = 1.68389689259e-08
Moon x = 383999886.965, y = 112.863376329
dX = 383999886.793, dY = 112.863376312
t = 11.239999999999805
Earth x = 0.172259293999, y = 1.68840130025e-08
Moon x = 383999886.864, y = 112.963878
dX = 383999886.692, dY = 112.963877983
t = 11.249999999999805
Earth x = 0.172566078169, y = 1.69291373358e-08
Moon x = 383999886.763, y = 113.064379671
dX = 383

...

Moon x = 383999710.133, y = 288.741300022
dX = 383999709.007, dY = 288.74129974
t = 28.740000000001693
Earth x = 1.12683186221, y = 2.82482154223e-07
Moon x = 383999710.031, y = 288.841801693
dX = 383999708.905, dY = 288.84180141
t = 28.750000000001695
Earth x = 1.12761629113, y = 2.82777174929e-07
Moon x = 383999709.93, y = 288.942303363
dX = 383999708.802, dY = 288.94230308
t = 28.760000000001696
Earth x = 1.12840099299, y = 2.83072400975e-07
Moon x = 383999709.829, y = 289.042805034
dX = 383999708.7, dY = 289.042804751
t = 28.770000000001698
Earth x = 1.12918596779, y = 2.83367832432e-07
Moon x = 383999709.728, y = 289.143306704
dX = 383999708.598, dY = 289.143306421
t = 28.7800000000017
Earth x = 1.12997121553, y = 2.8366346937e-07
Moon x = 383999709.626, y = 289.243808375
dX = 383999708.496, dY = 289.243808091
t = 28.7900000000017
Earth x = 1.13075673622, y = 2.83959311862e-07
Moon x = 383999709.525, y = 289.344310045
dX = 383999708.394, dY = 289.344309761
t = 28.800000000001702
E

Earth x = 1.43924611738, y = 4.07760358361e-07
Moon x = 383999672.131, y = 326.429426459
dX = 383999670.692, dY = 326.429426051
t = 32.490000000002105
Earth x = 1.44013262587, y = 4.08137158564e-07
Moon x = 383999672.03, y = 326.529928129
dX = 383999670.59, dY = 326.529927721
t = 32.5000000000021
Earth x = 1.4410194073, y = 4.08514190822e-07
Moon x = 383999671.929, y = 326.6304298
dX = 383999670.488, dY = 326.630429391
t = 32.5100000000021
Earth x = 1.44190646166, y = 4.08891455208e-07
Moon x = 383999671.827, y = 326.73093147
dX = 383999670.385, dY = 326.730931061
t = 32.5200000000021
Earth x = 1.44279378897, y = 4.09268951792e-07
Moon x = 383999671.726, y = 326.831433141
dX = 383999670.283, dY = 326.831432731
t = 32.5300000000021
Earth x = 1.44368138922, y = 4.09646680647e-07
Moon x = 383999671.624, y = 326.931934811
dX = 383999670.181, dY = 326.931934402
t = 32.540000000002095
Earth x = 1.44456926241, y = 4.10024641843e-07
Moon x = 383999671.523, y = 327.032436482
dX = 383999670.078,

Moon x = 383999634.599, y = 363.615044511
dX = 383999632.813, dY = 363.615043948
t = 36.19000000000137
Earth x = 1.786874007, y = 5.64083445293e-07
Moon x = 383999634.498, y = 363.715546182
dX = 383999632.711, dY = 363.715545618
t = 36.20000000000137
Earth x = 1.78786177625, y = 5.64551241139e-07
Moon x = 383999634.396, y = 363.816047852
dX = 383999632.608, dY = 363.816047288
t = 36.210000000001365
Earth x = 1.78884981844, y = 5.65019295543e-07
Moon x = 383999634.295, y = 363.916549522
dX = 383999632.506, dY = 363.916548957
t = 36.22000000000136
Earth x = 1.78983813357, y = 5.65487608577e-07
Moon x = 383999634.193, y = 364.017051193
dX = 383999632.403, dY = 364.017050627
t = 36.23000000000136
Earth x = 1.79082672164, y = 5.65956180312e-07
Moon x = 383999634.092, y = 364.117552863
dX = 383999632.301, dY = 364.117552297
t = 36.24000000000136
Earth x = 1.79181558266, y = 5.66425010819e-07
Moon x = 383999633.99, y = 364.218054534
dX = 383999632.198, dY = 364.218053967
t = 36.25000000000136

Moon x = 383999594.794, y = 403.011699274
dX = 383999592.6, dY = 403.011698507
t = 40.11000000000059
Earth x = 2.19499659227, y = 7.67985966131e-07
Moon x = 383999594.693, y = 403.112200945
dX = 383999592.498, dY = 403.112200177
t = 40.12000000000059
Earth x = 2.19609135405, y = 7.68560591279e-07
Moon x = 383999594.591, y = 403.212702615
dX = 383999592.395, dY = 403.212701846
t = 40.130000000000585
Earth x = 2.19718638877, y = 7.69135502988e-07
Moon x = 383999594.49, y = 403.313204285
dX = 383999592.292, dY = 403.313203516
t = 40.14000000000058
Earth x = 2.19828169643, y = 7.69710701329e-07
Moon x = 383999594.388, y = 403.413705955
dX = 383999592.19, dY = 403.413705186
t = 40.15000000000058
Earth x = 2.19937727703, y = 7.70286186374e-07
Moon x = 383999594.286, y = 403.514207626
dX = 383999592.087, dY = 403.514206855
t = 40.16000000000058
Earth x = 2.20047313057, y = 7.70861958193e-07
Moon x = 383999594.185, y = 403.614709296
dX = 383999591.984, dY = 403.614708525
t = 40.17000000000058


KeyboardInterrupt: 

Класс для построения анимации движения тел относительно Солнца:

In [13]:
import math
from turtle import *

G = 6.67428e-11
AU = (149.6e6 * 1000)
SCALE = 250 / AU

class Body(Turtle):

    name = 'Body'
    mass = None
    vx = vy = 0.0
    px = py = 0.0
    
    def attraction(self, other):
        if self is other:
            raise ValueError("Attraction of object %r to itself requested"
                             % self.name)

        
        sx, sy = self.px, self.py
        ox, oy = other.px, other.py
        dx = (ox-sx)
        dy = (oy-sy)
        d = math.sqrt(dx**2 + dy**2)

        if d == 0:
            raise ValueError("Collision between objects %r and %r"
                             % (self.name, other.name))
        
        f = G * self.mass * other.mass / (d**2)

        theta = math.atan2(dy, dx)
        fx = math.cos(theta) * f
        fy = math.sin(theta) * f
        return fx, fy

def update_info(step, bodies):
    print('Step #{}'.format(step))
    for body in bodies:
        s = '{:<8}  Pos.={:>6.2f} {:>6.2f} Vel.={:>10.3f} {:>10.3f}'.format(
            body.name, body.px/AU, body.py/AU, body.vx, body.vy)
        print(s)
    print()

def loop(bodies):
    timestep = 24*3600  # 1 day
    
    for body in bodies:
        body.penup()
        body.hideturtle()

    step = 1
    while True:
        update_info(step, bodies)
        step += 1

        force = {}
        for body in bodies:
            total_fx = total_fy = 0.0
            for other in bodies:
                if body is other:
                    continue
                fx, fy = body.attraction(other)
                total_fx += fx
                total_fy += fy

            force[body] = (total_fx, total_fy)

        for body in bodies:
            fx, fy = force[body]
            body.vx += fx / body.mass * timestep
            body.vy += fy / body.mass * timestep
            body.px += body.vx * timestep
            body.py += body.vy * timestep
            body.goto(body.px*SCALE, body.py*SCALE)
            body.dot(3)


def main():
    sun = Body()
    sun.name = 'Sun'
    sun.mass = 1.98892 * 10**30
    sun.pencolor('yellow')

    earth = Body()
    earth.name = 'Earth'
    earth.mass = 5.9742 * 10**24
    earth.px = -1*AU
    earth.vy = 29.783 * 1000            # 29.783 km/sec
    earth.pencolor('blue')

    venus = Body()
    venus.name = 'Venus'
    venus.mass = 4.8685 * 10**24
    venus.px = 0.723 * AU
    venus.vy = -35.02 * 1000
    venus.pencolor('red')

    loop([sun, earth, venus])

In [ ]:
main()

Скриншоты работы программы:

![T_start](shm1.png)

![T_end](shm2.png)

Программа для построения анимации движения Земли и Венеры относительно Солнца(необходима установка библиотек Tkinter и Turtle https://yadi.sk/d/c5iK3NTK3RsQt4